# Evaluate LinODEnet on Final Product Task

In [1]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

import logging
logging.basicConfig(level=logging.INFO)

In [2]:
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=4, floatmode='fixed', suppress=True)
rng = np.random.default_rng()

In [8]:
from tsdm.tasks import KIWI_FINAL_PRODUCT
task = KIWI_FINAL_PRODUCT(target="OD600")

# Load Model

In [6]:
import torch
from torch import jit
from torchinfo import summary
PATH = "checkpoints/LinODEnet/KIWI_RUNS/"
NAME = "SurrLoss+Sequential_Filter/2021-12-20T13:43:00/LinODEnet-70"
model = torch.jit.load(PATH + NAME, torch.device("cpu"))
summary(model, depth=1)